In [17]:
%reset
#%matplotlib qt

In [18]:
import matplotlib.pyplot as plt
import numpy as np
import scipy.io
import scipy.stats
import pandas as pd
import mne
from mne.time_frequency import tfr_morlet
from mne.stats import permutation_cluster_1samp_test
from mne.stats import permutation_cluster_test
from mne import io
import pickle
from os.path import exists
import gc
from mne import stats

gc.collect()

0

In [ ]:
epochs_dir = 'C:\AnatArzData\YKM_data\epochs_and_evoked_allSubs'
prepro_name = "referenced"
import_type = "5Electorodes_plainEEGLAB"
fig_output_dir = "C:\\Users\Sharon\OneDrive\Desktop\AnatArzi\scripts\Python_scripts - Sharons\mycode\outputs\\timefreq_clusterPerm"

## Import epochs

In [20]:
with open(f'{epochs_dir}\\{import_type}.pkl', 'rb') as file:
    [allEvents_df, allEpochs_perCond, cfg] = pickle.load(file)

cfg['sample_freq'] = 250

max_freq  =cfg['sample_freq']/2
times = cfg['times']
time0_i = np.where(times==0)[0][0]
hz_y = np.fft.rfftfreq(len(times[time0_i:]), 1.0/cfg['sample_freq'])

In [21]:
allEvents_df

,Cond_id,Subject,Vigilance,TOA,ToneHeight,TrialPosition,SamplesAmount,Epoch_eeglab_ids,TOA_cond
0,0,08,Wn,T,650,0,24,"[142, 262, 422, 682, 942, 1062, 1262, 1582, 16...",T
1,1,08,Wn,T,650,1,24,"[144, 264, 424, 684, 944, 1064, 1264, 1584, 16...",T
2,2,08,Wn,T,650,2,24,"[146, 266, 426, 686, 946, 1066, 1266, 1586, 16...",T
3,3,08,Wn,T,650,3,24,"[148, 268, 428, 688, 948, 1068, 1268, 1588, 16...",T
4,4,08,Wn,T,650,4,24,"[150, 270, 430, 690, 950, 1070, 1270, 1590, 16...",T
...,...,...,...,...,...,...,...,...,...
30235,30235,38,REM,A,5302,5,0,[],A
30236,30236,38,REM,A,5302,6,0,[],A
30237,30237,38,REM,A,5302,7,0,[],A
30238,30238,38,REM,A,5302,8,0,[],A


In [22]:
example_subject = '32'
example_epochs_filename = f"epochs_Wn_s{example_subject}_file"
data_path = 'C:\AnatArzData\Data\\reref'
subject_setfile_wake_n = data_path + '\s_32_wake_night_referenced.set'

if exists(example_epochs_filename):
    with open(example_epochs_filename, 'rb') as config_dictionary_file:
        epochs_Wn_example_sub = pickle.load(config_dictionary_file)
        #print(epochs_Wn_example_sub)
else:
    epochs_Wn_example_sub = io.read_epochs_eeglab(subject_setfile_wake_n, events=None, event_id=None,eog=(),verbose=None, uint16_codec=None)
    with open(example_epochs_filename, 'wb') as epochs_Wn_s_example_file:
        pickle.dump(epochs_Wn_example_sub, epochs_Wn_s_example_file)

montage = mne.channels.make_standard_montage('GSN-HydroCel-128')
epochs_Wn_example_sub_piked = epochs_Wn_example_sub.pick_channels(cfg['ch_names'])
epochs_Wn_example_sub_monatged = epochs_Wn_example_sub_piked.set_montage(montage)
info = epochs_Wn_example_sub_monatged.info

In [47]:
def getEpochsPerCond(cond_df,dataset):
    df_minTrials = cond_df[(cond_df.SamplesAmount > 0)] # discard cond with 0 enough samples
    keys = (str(key) for key in df_minTrials.Cond_id)
    epochs = {str_key: dataset[str_key] for str_key in keys}
    return df_minTrials, epochs

# output: [#conds, #elect, #y_ax]
def getEvokedPerCondAndElectd(constraints,dataset,y_ax,outputType='array',tmin=-0.1):
    curr_df =allEvents_df.copy()
    # apply constraints
    for key in constraints: curr_df = curr_df[(curr_df[key] == constraints[key])]

    conds_df, epochsPerCond = getEpochsPerCond(curr_df,dataset)
    evoked_perCond_andElectd = np.zeros((len(epochsPerCond),np.size(cfg['electrodes']),len(y_ax)))

    for cond_i, cond in enumerate(epochsPerCond):
        evoked_perCond_andElectd[cond_i] = np.squeeze(np.nanmean(epochsPerCond[cond],axis=2))

    if outputType =='array':
        return conds_df, evoked_perCond_andElectd
    elif outputType =='mne':
        # Epochs array shapr should be: (len(epochs_samps_per_cond),len(electrodes),len(y_ax))
        mne_epochs = mne.EpochsArray(evoked_perCond_andElectd, info,tmin=tmin,baseline=baseline)
        return conds_df, mne_epochs

def applyDesign(ax,title=''):
    ax.set_facecolor('silver')
    ax.set_title(title)
    ax.legend(loc = 'upper right',prop={'size': 10})
    ax.axvline(x=0,color='gray', linestyle='--',label ="_nolegend_")
    ax.axhline(y=0, color='gray', linestyle='-',label ="_nolegend_")
    ax.set_ylabel('magnitude')
    ax.tick_params(axis='both', which='major', labelsize=14)
    ax.set_xlabel('Time (ms)')

## Time-freq functions

In [48]:
cond_1 = {'TOA_cond':'T','Vigilance':'Wn'}
cond_2 = {'TOA_cond':'T','Vigilance':'N2'}
contrast_name = "allSubs_T_WnvsN3"

#### Parameters:
# ==================
# zscore baseline
## Tail is 0, so the statistic is thresholded on both sides of the distribution.
baseline = (None, 0) ## For timefreq analysis
p_value = 0.05 # default 0.05 # for clusters
decim = 1# default 2 for testing.. For reals - 1
n_permutations=100 # default 1k for testing. For reals - 10K
min_freq = 4
freqs = np.arange(min_freq,50, 1)  # define frequencies of interest
n_cycles = freqs / min_freq # different number of cycle per frequency

tail = 0 # 0 = two-tailed test (for clusters (as we want positive and negative clusters))
p_value_pixels = 0.05

In [55]:
def plot_clusters_map(ax1,T_obs,ch_idx,tfr_epochs,t_thresh,T_obs_plot):
    vmax = np.max(np.abs(T_obs))
    vmin = -vmax
    ax1.imshow(T_obs[ch_idx], cmap=plt.cm.gray,
               extent=[times[0], times[-1], freqs[0], freqs[-1]],
               aspect='auto', origin='lower', vmin=vmin, vmax=vmax)
    ax1.imshow(T_obs_plot[ch_idx], cmap=plt.cm.RdBu_r,
               extent=[times[0], times[-1], freqs[0], freqs[-1]],
               aspect='auto', origin='lower', vmin=vmin, vmax=vmax)
    #ax1.set_colorbar()
    ax1.set_xlabel('Time (ms)')
    ax1.set_ylabel('Frequency (Hz)')
    ax1.set_title(f'Induced power ({tfr_epochs.ch_names[ch_idx]})\nThreshold:{t_thresh}\n cluster p_val={p_value}\ncontrast:{cond_1};{cond_2}')
def plot_elecds_erps(ax2,mean_epochs_time_diff,ch_idx,tfr_epochs):
    ax2.plot(times,mean_epochs_time_diff.T,color='blue')
    mean_electrods = np.mean(mean_epochs_time_diff,axis=0)
    ax2.plot(times,mean_electrods,color='yellow',label='mean')
    ax2.plot(times,mean_epochs_time_diff[ch_idx,:],label=tfr_epochs.ch_names[ch_idx],color='red')
    ax2.legend()
    ax2.set_xlim(times[0],times[-1])
    applyDesign(ax2,'ERP difference')

In [56]:
def a(cond_1,cond_2, contrast_name):
    epochs_time_diff = []
    epochs_power_diff = []
    for sub in cfg['subs']:
        currContr_conds1 = cond_1.copy()
        currContr_conds2 = cond_2.copy()
        currContr_conds1['Subject'] = sub
        currContr_conds2['Subject'] = sub
        __, cont_epochs1 = getEvokedPerCondAndElectd(currContr_conds1,allEpochs_perCond,times, outputType='mne')
        __, cont_epochs2 = getEvokedPerCondAndElectd(currContr_conds2,allEpochs_perCond,times, outputType='mne')
        tfr_epochs1 = tfr_morlet(cont_epochs1, freqs, n_cycles=n_cycles, decim=decim, average=True, return_itc=False)
        tfr_epochs2 = tfr_morlet(cont_epochs2, freqs, n_cycles=n_cycles, decim=decim, average=True, return_itc=False)
        ## TODO: The baseline maybe should be applied together for both conditions
        tfr_epochs1.apply_baseline(mode='zscore', baseline=baseline)
        tfr_epochs2.apply_baseline(mode='zscore', baseline=baseline)
        epochs_power1 = tfr_epochs1.data # elec, freqs, time
        epochs_power2 = tfr_epochs2.data # elec, freqs, time
        means_diff = epochs_power1-epochs_power2
        epochs_power_diff.append(means_diff)

        epochs_time_diff.append(np.mean(cont_epochs1.get_data(),axis=0) - np.mean(cont_epochs2.get_data(),axis=0))

    # (n_epochs, n_channels, n_freqs, n_times)
    epochs_power_diff_arr = np.zeros((len(cfg['subs']),len(cfg['electrodes']),len(freqs),len(times)))
    for s,subject in enumerate(cfg['subs']):
        epochs_power_diff_arr[s,:,:,:] = epochs_power_diff[s]

    #### Define adjacency for statistics
    tfr_epochs = tfr_epochs1
    sensor_adjacency, ch_names = mne.channels.find_ch_adjacency(tfr_epochs.info,ch_type=None)
    use_idx = [ch_names.index(ch_name) for ch_name in tfr_epochs.ch_names]
    sensor_adjacency = sensor_adjacency[use_idx][:, use_idx]
    assert sensor_adjacency.shape == (len(tfr_epochs.ch_names), len(tfr_epochs.ch_names))
    assert epochs_power_diff_arr.shape == (len(cfg['subs']), len(tfr_epochs.ch_names), len(tfr_epochs.freqs), len(tfr_epochs.times))
    adjacency = mne.stats.combine_adjacency(sensor_adjacency, len(tfr_epochs.freqs), len(tfr_epochs.times))
    assert adjacency.shape[0] == adjacency.shape[1] == len(tfr_epochs.ch_names) * len(tfr_epochs.freqs) * len(tfr_epochs.times)

    ### run cluster permutation
    degrees_of_freedom = len(cfg['subs']) - 1
    t_thresh = scipy.stats.t.ppf(1 - p_value_pixels / 2, df=degrees_of_freedom)
    T_obs, clusters, cluster_p_values, H0 = permutation_cluster_1samp_test(epochs_power_diff_arr, n_permutations=n_permutations, threshold=t_thresh, tail=tail,  out_type='mask', verbose=True)

    ############# plot
    T_obs_plot = np.nan * np.ones_like(T_obs)
    for c, clust_p_val in zip(clusters, cluster_p_values):
        if clust_p_val <= p_value:
            T_obs_plot[c] = T_obs[c]
    mean_epochs_time_diff = np.mean(epochs_time_diff,axis=0)

    for ch_idx, elecd in enumerate(cfg['electrodes']):
        fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(10, 8))
        fig.subplots_adjust(left=0.12,bottom= 0.08,right= 0.96, top=0.85, wspace=0.2, hspace=0.3)
        plot_clusters_map(ax1,T_obs,ch_idx,tfr_epochs,t_thresh,T_obs_plot)
        plot_elecds_erps(ax2,mean_epochs_time_diff,ch_idx,tfr_epochs)
        plt.ioff()
        plt.savefig(f'{fig_output_dir}/timeFreq_clusters_{contrast_name}_{tfr_epochs.ch_names[ch_idx]}.png',bbox_inches='tight')

a(cond_1,cond_2, contrast_name)

Not setting metadata
89 matching events found
Setting baseline interval to [-0.1, 0.0] sec
Applying baseline correction (mode: mean)
0 projection items activated
0 bad epochs dropped
Not setting metadata
90 matching events found
Setting baseline interval to [-0.1, 0.0] sec
Applying baseline correction (mode: mean)
0 projection items activated
0 bad epochs dropped
Applying baseline correction (mode: zscore)
Applying baseline correction (mode: zscore)
Not setting metadata
89 matching events found
Setting baseline interval to [-0.1, 0.0] sec
Applying baseline correction (mode: mean)
0 projection items activated
0 bad epochs dropped
Not setting metadata
90 matching events found
Setting baseline interval to [-0.1, 0.0] sec
Applying baseline correction (mode: mean)
0 projection items activated
0 bad epochs dropped
Applying baseline correction (mode: zscore)
Applying baseline correction (mode: zscore)
Not setting metadata
89 matching events found
Setting baseline interval to [-0.1, 0.0] sec
A

  0%|          |  : 0/99 [00:00<?,       ?it/s]

Computing cluster p-values
Done.


## Run many contrasts at once

In [59]:
%matplotlib qt

contrasts = {}
contrasts["allSubs_RandvsFixed_Wn"] = {'cond_1':{'TOA_cond':'Rand','Vigilance':'Wn'},'cond_2':{'TOA_cond':'Fixed','Vigilance':'Wn'}}
contrasts["allSubs_RandvsFixed_N2"] = {'cond_1':{'TOA_cond':'Rand','Vigilance':'N2'},'cond_2':{'TOA_cond':'Fixed','Vigilance':'N2'}}
contrasts["allSubs_RandvsFixed_N3"] = {'cond_1':{'TOA_cond':'Rand','Vigilance':'N3'},'cond_2':{'TOA_cond':'Fixed','Vigilance':'N3'}}
contrasts["allSubs_RandvsFixed_REM"] = {'cond_1':{'TOA_cond':'Rand','Vigilance':'REM'},'cond_2':{'TOA_cond':'Fixed','Vigilance':'REM'}}
contrasts["allSubs_T_WnvsN2"] = {'cond_1':{'TOA_cond':'T','Vigilance':'Wn'},'cond_2':{'TOA_cond':'T','Vigilance':'N2'}}
contrasts['allSubs_T_WnvsN3'] = {'cond_1':{'TOA_cond':'T','Vigilance':'Wn'},'cond_2':{'TOA_cond':'T','Vigilance':'N3'}}
contrasts['allSubs_T_WnvsREM'] = {'cond_1':{'TOA_cond':'T','Vigilance':'Wn'},'cond_2':{'TOA_cond':'T','Vigilance':'REM'}}
contrasts['allSubs_TvsRand_Wn'] = {'cond_1':{'TOA_cond':'T','Vigilance':'Wn'},'cond_2':{'TOA_cond':'Rand','Vigilance':'Wn'}}
contrasts['allSubs_TvsFixed_Wn'] = {'cond_1':{'TOA_cond':'T','Vigilance':'Wn'},'cond_2':{'TOA_cond':'Fixed','Vigilance':'Wn'}}
contrasts['allSubs_TvsRand_N2'] = {'cond_1':{'TOA_cond':'T','Vigilance':'N2'},'cond_2':{'TOA_cond':'Rand','Vigilance':'N2'}}
contrasts['allSubs_TvsFixed_N2'] = {'cond_1':{'TOA_cond':'T','Vigilance':'N2'},'cond_2':{'TOA_cond':'Fixed','Vigilance':'N2'}}
contrasts['allSubs_TvsRand_N3'] = {'cond_1':{'TOA_cond':'T','Vigilance':'N3'},'cond_2':{'TOA_cond':'Rand','Vigilance':'N3'}}
contrasts['allSubs_TvsFixed_N3'] = {'cond_1':{'TOA_cond':'T','Vigilance':'N3'},'cond_2':{'TOA_cond':'Fixed','Vigilance':'N3'}}
contrasts['allSubs_TvsRand_REM'] = {'cond_1':{'TOA_cond':'T','Vigilance':'REM'},'cond_2':{'TOA_cond':'Rand','Vigilance':'REM'}}
contrasts['allSubs_TvsFixed_REM'] = {'cond_1':{'TOA_cond':'T','Vigilance':'REM'},'cond_2':{'TOA_cond':'Fixed','Vigilance':'REM'}}

for contrast in contrasts:
    cond_1 = contrasts[contrast]['cond_1']
    cond_2 = contrasts[contrast]['cond_2']
    print(f'Now computeing {cond_1} and {cond_2}')
    contrast_name = contrast.title()
    a(cond_1, cond_2, contrast_name)

Now computeing {'TOA_cond': 'Rand', 'Vigilance': 'Wn'} and {'TOA_cond': 'Fixed', 'Vigilance': 'Wn'}
Not setting metadata
25 matching events found
Setting baseline interval to [-0.1, 0.0] sec
Applying baseline correction (mode: mean)
0 projection items activated
0 bad epochs dropped
Not setting metadata
9 matching events found
Setting baseline interval to [-0.1, 0.0] sec
Applying baseline correction (mode: mean)
0 projection items activated
0 bad epochs dropped
Applying baseline correction (mode: zscore)
Applying baseline correction (mode: zscore)
Not setting metadata
27 matching events found
Setting baseline interval to [-0.1, 0.0] sec
Applying baseline correction (mode: mean)
0 projection items activated
0 bad epochs dropped
Not setting metadata
9 matching events found
Setting baseline interval to [-0.1, 0.0] sec
Applying baseline correction (mode: mean)
0 projection items activated
0 bad epochs dropped
Applying baseline correction (mode: zscore)
Applying baseline correction (mode: zs

  0%|          |  : 0/99 [00:00<?,       ?it/s]

Computing cluster p-values
Done.
Now computeing {'TOA_cond': 'Rand', 'Vigilance': 'N2'} and {'TOA_cond': 'Fixed', 'Vigilance': 'N2'}
Not setting metadata
27 matching events found
Setting baseline interval to [-0.1, 0.0] sec
Applying baseline correction (mode: mean)
0 projection items activated
0 bad epochs dropped
Not setting metadata
9 matching events found
Setting baseline interval to [-0.1, 0.0] sec
Applying baseline correction (mode: mean)
0 projection items activated
0 bad epochs dropped
Applying baseline correction (mode: zscore)
Applying baseline correction (mode: zscore)
Not setting metadata
27 matching events found
Setting baseline interval to [-0.1, 0.0] sec
Applying baseline correction (mode: mean)
0 projection items activated
0 bad epochs dropped
Not setting metadata
9 matching events found
Setting baseline interval to [-0.1, 0.0] sec
Applying baseline correction (mode: mean)
0 projection items activated
0 bad epochs dropped
Applying baseline correction (mode: zscore)
Apply

  0%|          |  : 0/99 [00:00<?,       ?it/s]

Computing cluster p-values
Done.
Now computeing {'TOA_cond': 'Rand', 'Vigilance': 'N3'} and {'TOA_cond': 'Fixed', 'Vigilance': 'N3'}
Not setting metadata
25 matching events found
Setting baseline interval to [-0.1, 0.0] sec
Applying baseline correction (mode: mean)
0 projection items activated
0 bad epochs dropped
Not setting metadata
9 matching events found
Setting baseline interval to [-0.1, 0.0] sec
Applying baseline correction (mode: mean)
0 projection items activated
0 bad epochs dropped
Applying baseline correction (mode: zscore)
Applying baseline correction (mode: zscore)
Not setting metadata
27 matching events found
Setting baseline interval to [-0.1, 0.0] sec
Applying baseline correction (mode: mean)
0 projection items activated
0 bad epochs dropped
Not setting metadata
9 matching events found
Setting baseline interval to [-0.1, 0.0] sec
Applying baseline correction (mode: mean)
0 projection items activated
0 bad epochs dropped
Applying baseline correction (mode: zscore)
Apply

...

Computing cluster p-values
Done.
Now computeing {'TOA_cond': 'T', 'Vigilance': 'REM'} and {'TOA_cond': 'Rand', 'Vigilance': 'REM'}
Not setting metadata
90 matching events found
Setting baseline interval to [-0.1, 0.0] sec
Applying baseline correction (mode: mean)
0 projection items activated
0 bad epochs dropped
Not setting metadata
22 matching events found
Setting baseline interval to [-0.1, 0.0] sec
Applying baseline correction (mode: mean)
0 projection items activated
0 bad epochs dropped
Applying baseline correction (mode: zscore)
Applying baseline correction (mode: zscore)
Not setting metadata
90 matching events found
Setting baseline interval to [-0.1, 0.0] sec
Applying baseline correction (mode: mean)
0 projection items activated
0 bad epochs dropped
Not setting metadata
26 matching events found
Setting baseline interval to [-0.1, 0.0] sec
Applying baseline correction (mode: mean)
0 projection items activated
0 bad epochs dropped
Applying baseline correction (mode: zscore)
Apply

  0%|          |  : 0/99 [00:00<?,       ?it/s]

Computing cluster p-values
Done.
Now computeing {'TOA_cond': 'T', 'Vigilance': 'REM'} and {'TOA_cond': 'Fixed', 'Vigilance': 'REM'}
Not setting metadata
90 matching events found
Setting baseline interval to [-0.1, 0.0] sec
Applying baseline correction (mode: mean)
0 projection items activated
0 bad epochs dropped
Not setting metadata
9 matching events found
Setting baseline interval to [-0.1, 0.0] sec
Applying baseline correction (mode: mean)
0 projection items activated
0 bad epochs dropped
Applying baseline correction (mode: zscore)
Applying baseline correction (mode: zscore)
Not setting metadata
90 matching events found
Setting baseline interval to [-0.1, 0.0] sec
Applying baseline correction (mode: mean)
0 projection items activated
0 bad epochs dropped
Not setting metadata
9 matching events found
Setting baseline interval to [-0.1, 0.0] sec
Applying baseline correction (mode: mean)
0 projection items activated
0 bad epochs dropped
Applying baseline correction (mode: zscore)
Applyi

  0%|          |  : 0/99 [00:00<?,       ?it/s]

Computing cluster p-values
Done.
